In [1]:
#Had to modify the code since I was getting errors on my computer


# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
#from keras import backend as K
#K.set_image_dim_ordering('th')

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 5
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

ImportError: cannot import name 'maxnorm' from 'keras.constraints' (/Users/enochjoy/anaconda3/lib/python3.11/site-packages/keras/constraints/__init__.py)

In [4]:
# Simple CNN model for CIFAR-10
import numpy
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD  # Using the updated import path
from tensorflow.keras.optimizers import legacy  # Importing the legacy optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=MaxNorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=MaxNorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
epochs = 5
lrate = 0.01
decay = lrate / epochs
# Use the legacy version of SGD as suggested by the warning
sgd = legacy.SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Print model summary
print(model.summary())

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               4194816   
                                                      

In [7]:
#1) With Adjusted Hyperparameters, Advanced Optimizers, Data Augmentation, and Regularization Techniques


import numpy as np
import math
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers.legacy import Adam  # Updated import path for legacy Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Load and preprocess the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model architecture with added layers, batch normalization, and adjusted dropout rates
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu', kernel_constraint=MaxNorm(3)),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)),
    BatchNormalization(),
    Dropout(0.3),
    Flatten(),
    Dense(512, activation='relu', kernel_constraint=MaxNorm(3)),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Learning rate scheduler function
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# Compile the model with the legacy Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Learning rate scheduler callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

# Train the model with data augmentation
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=X_train.shape[0] // 32, epochs=25,
          validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f%%' % (scores[1]*100))


Epoch 1/25
1562/1562 [==============================] - 142s 91ms/step - loss: 1.8248 - accuracy: 0.3532 - val_loss: 1.3980 - val_accuracy: 0.4791 - lr: 0.0010
Epoch 2/25
1562/1562 [==============================] - 154s 99ms/step - loss: 1.4374 - accuracy: 0.4804 - val_loss: 1.7104 - val_accuracy: 0.4803 - lr: 0.0010
Epoch 3/25
1562/1562 [==============================] - 150s 96ms/step - loss: 1.2938 - accuracy: 0.5421 - val_loss: 3.3843 - val_accuracy: 0.3870 - lr: 0.0010
Epoch 4/25
1562/1562 [==============================] - 153s 98ms/step - loss: 1.2038 - accuracy: 0.5761 - val_loss: 1.1907 - val_accuracy: 0.6005 - lr: 0.0010
Epoch 5/25
1562/1562 [==============================] - 155s 99ms/step - loss: 1.1339 - accuracy: 0.6002 - val_loss: 0.9587 - val_accuracy: 0.6656 - lr: 0.0010
Epoch 6/25
1562/1562 [==============================] - 156s 100ms/step - loss: 1.0973 - accuracy: 0.6158 - val_loss: 1.0638 - val_accuracy: 0.6475 - lr: 0.0010
Epoch 7/25
1562/1562 [=================

In [ ]:
#2 2. Provide logical description of which steps lead to improved response and what was its impact on 
#architecture behavior.

#Improving the response of a convolutional neural network (CNN) on image classification tasks, like CIFAR-10, 
#involves a combination of tuning hyperparameters, adjusting the model architecture, and employing specific data 
#preprocessing techniques. The modifications introduced in the provided code aimed to enhance the model's ability 
#to generalize from the training data and perform more accurately on unseen test data.
#Here's a logical description of the steps taken and their impact on the model's behavior:

#1. Switching to Legacy Adam Optimizer
#2. Adding Batch Normalization Layers
#3. Adjusting Dropout Rates
#4. Implementing Data Augmentation
#5. Learning Rate Scheduling
#6. Increasing Model Depth and Adjusting Filters